In [1]:
import os
import numpy as np
import re
import shutil
import tensorflow as tf
DATA_DIR = './data'
CHEKPOINT_DIR = os.path.join(DATA_DIR,"chekpoints")


In [3]:
def download_and_read(urls):
    texts = []
    for i, url in enumerate(urls):
        p = tf.keras.utils.get_file("ex1-{:d}.txt".format(i), url,cache_dir=".")
        text = open(p,"r", encoding="utf8").read()
        text = text.replace("\ufeff","")
        text = text.replace("\n","")
        text = re.sub(r'\s+',"",text)
        texts.extend(text)
        
    return texts
texts = download_and_read([
    "http://www.gutenberg.org/cache/epub/28885/pg28885.txt",
    "https://www.gutenberg.org/files/12/12-0.txt"
])
        

196464/196464 [==============================] - 2s 12us/step


In [5]:
vocab = sorted(set(texts))
print("vocab size: {:d}".format(len(vocab)))

char2idx = {c:i for i,c in enumerate(vocab)}
idx2char = {i:c for c,i in char2idx.items()}

vocab size: 92


In [8]:
texts_as_ints = np.array([char2idx[c] for c in texts], )
data = tf.data.Dataset.from_tensor_slices(texts_as_ints)

In [14]:
seq_length = 100
sequences = data.batch(seq_length+1,drop_remainder=True)
def split_train_labels(sequence,):
    input_seq = sequence[0:-1]
    output_seq = sequence[1:]
    return input_seq,output_seq

sequences = sequences.map(split_train_labels)

batch_size = 64
steps_per_epoch = len(texts) // seq_length // batch_size

dataset = sequences.shuffle(10000).batch(batch_size, drop_remainder=True)
    

In [15]:
sequences

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))>

In [16]:
class CharGenModel(tf.keras.Model):
    def __init__(self,vocab_size,num_timesteps,embedding_dim,**kwargs):
        super(CharGenModel, self).__init__(**kwargs)
        self.embedding_layer = tf.keras.layers.Embedding(
            vocab_size, embedding_dim
        )
        self.rnn_layer = tf.keras.layers.GRU(
            num_timesteps,
            recurrent_initializer="glorot_uniform",
            recurrent_activation="sigmoid",
            stateful=True,
            return_sequences=True
        )
        self.dense_layer = tf.keras.layers.Dense(vocab_size)
    
    def call(self,x):
        x = self.embedding_layer(x)
        x = self.rnn_layer(x)
        x = self.dense_layer(x)
        return x
    
vocab_size = len(vocab)
embedding_dim = 256

model = CharGenModel(vocab_size,seq_length,embedding_dim)
model.build(input_shape=(batch_size,seq_length))

def loss(labels,predictions):
    return tf.losses.sparse_categorical_crossentropy(
        labels,
        predictions,from_logits=True
    )

model.compile(optimizer=tf.optimizers.Adam(),loss=loss)

In [17]:
def generate_text(model,prefix_string,char2idx,idx2char,num_chars_to_generate=1000,temperature=1.0):
    input = [char2idx[s] for s in prefix_string]
    input = tf.expand_dims(input,0)
    
    text_generated = []
    model.reset_states()
    
    for i in range(num_chars_to_generate):
        preds = model(input)
        preds = tf.squeeze(preds,0)/temperature
        pred_id = tf.random.categorical(preds,num_samples=1)[-1,0].numpy()
        text_generated.append(idx2char[pred_id])
        input = tf.expand_dims([pred_id],0)
    return prefix_string+"".join(text_generated)



In [19]:
num_epochs = 50
for i in range(num_epochs//10):
    model.fit(
        dataset.repeat(),
        epochs = 10,
        steps_per_epoch=steps_per_epoch
    )
    checkpoint_file = os.path.join(CHEKPOINT_DIR,"model_epoch_{:d}".format(i+1))
    model.save_weights(checkpoint_file)
    gen_model = CharGenModel(vocab_size,seq_length,embedding_dim)
    gen_model.load_weights(checkpoint_file)
    gen_model.build(input_shape=(1,seq_length))
    print("After epoch: {:d} ".format(i+1)*10)
    print(generate_text(gen_model,"Alice",char2idx,idx2char))
    print("---")

Epoch 1/10
44/44 [==============================] - 1s 15ms/step - loss: 1.7603
Epoch 2/10
44/44 [==============================] - 1s 10ms/step - loss: 1.7565
Epoch 3/10
44/44 [==============================] - 1s 10ms/step - loss: 1.7529
Epoch 4/10
44/44 [==============================] - 1s 10ms/step - loss: 1.7485
Epoch 5/10
44/44 [==============================] - 1s 10ms/step - loss: 1.7447
Epoch 6/10
44/44 [==============================] - 1s 10ms/step - loss: 1.7422
Epoch 7/10
44/44 [==============================] - 1s 10ms/step - loss: 1.7375
Epoch 8/10
44/44 [==============================] - 1s 10ms/step - loss: 1.7325
Epoch 9/10
44/44 [==============================] - 1s 10ms/step - loss: 1.7313
Epoch 10/10
44/44 [==============================] - 1s 11ms/step - loss: 1.7274
After epoch: 1 After epoch: 1 After epoch: 1 After epoch: 1 After epoch: 1 After epoch: 1 After epoch: 1 After epoch: 1 After epoch: 1 After epoch: 1 
AliceIstabeIbegansuchavegoneaworcamequietchilled